# Transformer Neural Network

In [10]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

In [3]:
df = pd.read_csv('Data/SVG_poly1.csv') # importing data for SVG polyorder = 1
df.drop(columns=['Unnamed: 0'], inplace=True) 
df.set_index("Year", inplace=True)

In [11]:
# Define the Positional Encoding module
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [12]:
# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_size, output_size, d_model, nhead, num_layers, dropout=0.5):
        super().__init__()
        self.d_model = d_model
        self.pos_encoder = PositionalEncoding(input_size, dropout)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.encoder = nn.Linear(input_size, d_model)
        self.decoder = nn.Linear(d_model, output_size)
        self.init_weights()

    def forward(self, src):
        src = self.encoder(src) * np.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.weight.data.uniform_(-initrange, initrange)

In [14]:
# Define a custom PyTorch dataset for sunspot data
class SunspotDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length
        self.scaler = MinMaxScaler()

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, index):
        seq = self.data[index:index+self.seq_length]
        label = self.data[index+self.seq_length]
        seq = self.scaler.fit_transform(seq.reshape(-1, 1)).squeeze()
        label = self.scaler.transform(label.reshape(-1, 1)).squeeze()
        return torch.FloatTensor(seq), torch.FloatTensor(label)

In [21]:
# Load and preprocess the sunspot data
limit = 70000
skips = 28 # 1 data point per luna cycle
train_data = df.iloc[:limit:skips] 
test_data = df.iloc[limit::skips]

In [22]:
# Define the hyperparameters and data loaders
seq_length = 24
batch_size = 32
num_epochs = 100
input_size = 1
output_size = 1
d_model = 64
nhead = 2
num_layers = 2
dropout = 0.1

In [24]:
# Create training and testing datasets and data loaders
train_dataset = SunspotDataset(train_data, seq_length)
test_dataset = SunspotDataset(test_data, seq_length)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Train model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerModel(seq_length, num_features).to(device)
for epoch in range(100):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    print(f'Epoch {epoch+1} training loss: {running_loss/len(train_dataset):.4f}')

NameError: name 'model' is not defined